Analysing Corona-out-2 file

In [9]:
import json
file_1 = "/Users/manasarthak/Documents/DBMS Project/corona-out-2"
lines_file_1 = 0
tweets_file_1 = 0
retweets_file_1 = 0
field_counts_file_1 = {}
url_counts_file_1 = 0
sample_tweet_file_1 = None
sample_retweet_file_1 = None

with open(file_1, "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            lines_file_1 += 1
            is_retweet = data['text'].startswith('RT')
            retweets_file_1 += is_retweet
            tweets_file_1 += not is_retweet
            if is_retweet and not sample_retweet_file_1:
                sample_retweet_file_1 = data
            elif not is_retweet and not sample_tweet_file_1:
                sample_tweet_file_1 = data
            for key in data.keys():
                field_counts_file_1[key] = field_counts_file_1.get(key, 0) + 1
            url_counts_file_1 += len(data.get('entities', {}).get('urls', []))
        except json.JSONDecodeError:
            continue

print(f'Summary Report for {file_1}:')
print('Number of lines:', lines_file_1)
print('Number of tweets:', tweets_file_1)
print('Number of retweets:', retweets_file_1)
print('Number of URLs:', url_counts_file_1)
print('\nField Counts:')
for field, count in field_counts_file_1.items():
    print(field, ':', count)

print('\nSample Tweet from file 1:')
print(json.dumps(sample_tweet_file_1, indent=4))
print('\nSample Retweet from file 1:')
print(json.dumps(sample_retweet_file_1, indent=4))


Summary Report for /Users/manasarthak/Documents/DBMS Project/corona-out-2:
Number of lines: 18518
Number of tweets: 7356
Number of retweets: 11162
Number of URLs: 4441

Field Counts:
created_at : 18518
id : 18518
id_str : 18518
text : 18518
source : 18518
truncated : 18518
in_reply_to_status_id : 18518
in_reply_to_status_id_str : 18518
in_reply_to_user_id : 18518
in_reply_to_user_id_str : 18518
in_reply_to_screen_name : 18518
user : 18518
geo : 18518
coordinates : 18518
place : 18518
contributors : 18518
retweeted_status : 11159
quoted_status_id : 3013
quoted_status_id_str : 3013
quoted_status : 2994
quoted_status_permalink : 2994
is_quote_status : 18518
quote_count : 18518
reply_count : 18518
retweet_count : 18518
favorite_count : 18518
entities : 18518
favorited : 18518
retweeted : 18518
filter_level : 18518
lang : 18518
timestamp_ms : 18518
display_text_range : 3183
extended_tweet : 2912
possibly_sensitive : 3449
extended_entities : 1084
withheld_in_countries : 81

Sample Tweet from

In [2]:
def check_for_duplicate_ids(filename):
    ids_seen = set()
    duplicate_ids = set()

    with open(filename, "r") as file:
        for line in file:
            try:
                tweet = json.loads(line)
                tweet_id = tweet.get('id')
                if tweet_id in ids_seen:
                    duplicate_ids.add(tweet_id)
                else:
                    ids_seen.add(tweet_id)
            except json.JSONDecodeError:
                continue  # Skipping lines that are not valid JSON

    return duplicate_ids

duplicates = check_for_duplicate_ids(file_1)

if duplicates:
    print(f"Found duplicates for the following tweet IDs: {duplicates}")
else:
    print("No duplicate tweet IDs found.")


Found duplicates for the following tweet IDs: {1249406404013764608, 1249407126847524864, 1249406332349882370, 1249407196334559235, 1249407921412333568, 1249405427361763333, 1249407955298115587, 1249404402139570183, 1249408006544134145, 1249408330784739328, 1249408430382755840, 1249408655671386115}


In [7]:
def find_first_duplicate_for_id(filename, tweet_id_to_find):
    ids_seen = {}
    first_duplicate_line = None

    with open(filename, "r") as file:
        for line_number, line in enumerate(file, 1):
            try:
                tweet = json.loads(line)
                tweet_id = tweet.get('id')
                if tweet_id == tweet_id_to_find:
                    if tweet_id in ids_seen and first_duplicate_line is None:
                        # Store the duplicate line and stop searching
                        first_duplicate_line = (line_number, line)
                        break
                    else:
                        ids_seen[tweet_id] = (line_number, line)
            except json.JSONDecodeError:
                continue 

    return ids_seen[tweet_id_to_find], first_duplicate_line


tweet_id_to_find = 1249406404013764608  # Replace with the ID you want to check

original, duplicate = find_first_duplicate_for_id(file_1, tweet_id_to_find)

if duplicate:
    print(f"Original occurrence - Line {original[0]}: {json.dumps(json.loads(original[1]), indent=4)}")
    print(f"First duplicate - Line {duplicate[0]}: {json.dumps(json.loads(duplicate[1]), indent=4)}")
else:
    print(f"No duplicate found for Tweet ID: {tweet_id_to_find}")


Original occurrence - Line 19185: {
    "created_at": "Sun Apr 12 18:37:54 +0000 2020",
    "id": 1249406404013764608,
    "id_str": "1249406404013764608",
    "text": "Corona",
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 1248439943384440839,
        "id_str": "1248439943384440839",
        "name": "Wolf 3.0 \u26f3\ufe0f",
        "screen_name": "Asap_infinityy",
        "location": "910",
        "url": "http://Golfwang.com",
        "description": "Back on my bs // 21 // rip @Asap_infinity",
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 89,
        "friends_count": 171,
        "listed_count": 0,
        "favourites_count": 14

In [46]:
def find_tweet_with_id(file_name, tweet_id):
    with open(file_name, 'r') as file:
        for line in file:
            try:
                data = json.loads(line)
                if data.get("id") == tweet_id:
                    return data
            except json.JSONDecodeError:
                continue  # Skip invalid lines
    return None

# Tweet ID to find
target_tweet_id = 1249403770833981440

# Find and print the tweet
tweet = find_tweet_with_id(file_1, target_tweet_id)
if tweet:
    print(json.dumps(tweet, indent=4))
else:
    print(f"No tweet found with ID {target_tweet_id}.")

{
    "created_at": "Sun Apr 12 18:27:26 +0000 2020",
    "id": 1249403770833981440,
    "id_str": "1249403770833981440",
    "text": "RT @biannagolodryga: This image.\nThis quote.\n\n\u201cOne of the reasons the pandemic is hitting the country so hard is that no one appears to be\u2026",
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 270088915,
        "id_str": "270088915",
        "name": "Charles Mire",
        "screen_name": "charlesmire",
        "location": "Kitchener-Waterloo, Canada",
        "url": "http://www.structur3d.io",
        "description": "Avid outdoorsman, scientist, math nut....and I like beer. Co-founder of Structur3D Printing, tech for rubber additive manufacturing",
        

Analysing Corona-out-3 file

In [10]:
file_2 = "/Users/manasarthak/Documents/DBMS Project/corona-out-3"
lines_file_2 = 0
tweets_file_2 = 0
retweets_file_2 = 0
field_counts_file_2 = {}
url_counts_file_2 = 0
sample_tweet_file_2 = None
sample_retweet_file_2 = None

with open(file_2, "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            lines_file_2 += 1
            is_retweet = data['text'].startswith('RT')
            retweets_file_2 += is_retweet
            tweets_file_2 += not is_retweet
            if is_retweet and not sample_retweet_file_2:
                sample_retweet_file_2 = data
            elif not is_retweet and not sample_tweet_file_2:
                sample_tweet_file_2 = data
            for key in data.keys():
                field_counts_file_2[key] = field_counts_file_2.get(key, 0) + 1
            url_counts_file_2 += len(data.get('entities', {}).get('urls', []))
        except json.JSONDecodeError:
            continue

print(f'Summary Report for {file_2}:')
print('Number of lines:', lines_file_2)
print('Number of tweets:', tweets_file_2)
print('Number of retweets:', retweets_file_2)
print('Number of URLs:', url_counts_file_2)
print('\nField Counts:')
for field, count in field_counts_file_2.items():
    print(field, ':', count)

print('\nSample Tweet from file 2:')
print(json.dumps(sample_tweet_file_2, indent=4))
print('\nSample Retweet from file 2:')
print(json.dumps(sample_retweet_file_2, indent=4))


Summary Report for /Users/manasarthak/Documents/DBMS Project/corona-out-3:
Number of lines: 101916
Number of tweets: 40804
Number of retweets: 61112
Number of URLs: 32788

Field Counts:
created_at : 101916
id : 101916
id_str : 101916
text : 101916
display_text_range : 18124
source : 101916
truncated : 101916
in_reply_to_status_id : 101916
in_reply_to_status_id_str : 101916
in_reply_to_user_id : 101916
in_reply_to_user_id_str : 101916
in_reply_to_screen_name : 101916
user : 101916
geo : 101916
coordinates : 101916
place : 101916
contributors : 101916
is_quote_status : 101916
quote_count : 101916
reply_count : 101916
retweet_count : 101916
favorite_count : 101916
entities : 101916
extended_entities : 7267
favorited : 101916
retweeted : 101916
possibly_sensitive : 27433
filter_level : 101916
lang : 101916
timestamp_ms : 101916
retweeted_status : 61101
quoted_status_id : 16129
quoted_status_id_str : 16129
quoted_status : 16123
quoted_status_permalink : 16123
extended_tweet : 17760
withheld

In [5]:
duplicates = check_for_duplicate_ids(file_2)

if duplicates:
    print(f"Found duplicates for the following tweet IDs: {duplicates}")
else:
    print("No duplicate tweet IDs found.")


Found duplicates for the following tweet IDs: {1254023086535573504, 1254022835435196417, 1254023839706574848, 1254030389192151041, 1254030135457652736, 1254032422926245888, 1254033154689699842, 1254034659484237824, 1254034661417705472, 1254035939116093441, 1254041962597855233, 1254047745435934722, 1254049508675158021, 1254052393664249857, 1254052800348270599, 1254053307032768517, 1254048755235450880, 1254042462655520768, 1254047845151318016, 1254048253961740288, 1254053555348135936, 1254043217475108864}


In [34]:
tweet_id_to_find = 1254059748556771329  # Replace with the ID you want to check

original, duplicate = find_first_duplicate_for_id(file_2, tweet_id_to_find)

if duplicate:
    print(f"Original occurrence - Line {original[0]}: {json.dumps(json.loads(original[1]), indent=4)}")
    print(f"First duplicate - Line {duplicate[0]}: {json.dumps(json.loads(duplicate[1]), indent=4)}")
else:
    print(f"No duplicate found for Tweet ID: {tweet_id_to_find}")

No duplicate found for Tweet ID: 1254059748556771329


Some Stats about both files.

In [3]:
total_lines = lines_file_1 + lines_file_2
total_tweets = tweets_file_1 + tweets_file_2
total_retweets = retweets_file_1 + retweets_file_2
total_urls = url_counts_file_1 + url_counts_file_2
combined_field_counts = field_counts_file_1

for key, value in field_counts_file_2.items():
    combined_field_counts[key] = combined_field_counts.get(key, 0) + value

print('Total Summary Report:')
print('Number of lines:', total_lines)
print('Number of tweets:', total_tweets)
print('Number of retweets:', total_retweets)
print('Total number of URLs:', total_urls)
print('\nCombined Field Counts:')
for field, count in combined_field_counts.items():
    print(field, ':', count)

Total Summary Report:
Number of lines: 120434
Number of tweets: 48160
Number of retweets: 72274
Total number of URLs: 37229

Combined Field Counts:
created_at : 120434
id : 120434
id_str : 120434
text : 120434
source : 120434
truncated : 120434
in_reply_to_status_id : 120434
in_reply_to_status_id_str : 120434
in_reply_to_user_id : 120434
in_reply_to_user_id_str : 120434
in_reply_to_screen_name : 120434
user : 120434
geo : 120434
coordinates : 120434
place : 120434
contributors : 120434
retweeted_status : 72260
quoted_status_id : 19142
quoted_status_id_str : 19142
quoted_status : 19117
quoted_status_permalink : 19117
is_quote_status : 120434
quote_count : 120434
reply_count : 120434
retweet_count : 120434
favorite_count : 120434
entities : 120434
favorited : 120434
retweeted : 120434
filter_level : 120434
lang : 120434
timestamp_ms : 120434
display_text_range : 21307
extended_tweet : 20672
possibly_sensitive : 30882
extended_entities : 8351
withheld_in_countries : 127


Connecting with local Mysql

In [8]:
!pip install mysql-connector-python

In [1]:
import mysql.connector
from mysql.connector import Error

# Function to create a database connection
def create_database_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='twitter_datasets',
            user='root',
            password='JadonSancho@7'
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print("Error while connecting to MySQL", e)
        return None

In [3]:
def drop_and_create_tables(cursor):
    # Drop existing tables
    cursor.execute("DROP TABLE IF EXISTS Tweets")
    cursor.execute("DROP TABLE IF EXISTS Users")

    # Users Table
    users_table_query = """
    CREATE TABLE Users (
        user_id BIGINT PRIMARY KEY,
        name VARCHAR(255),
        screen_name VARCHAR(255),
        location VARCHAR(255),
        followers_count INT,
        friends_count INT,
        statuses_count INT,
        created_at DATETIME
    )
    """
    cursor.execute(users_table_query)

    # Tweets Table
    tweets_table_query = """
    CREATE TABLE Tweets (
        tweet_id BIGINT PRIMARY KEY,
        user_id BIGINT,
        created_at DATETIME,
        is_quote_status BOOLEAN,
        quote_count INT,
        reply_count INT,
        retweet_count INT,
        favorite_count INT,
        is_retweet BOOLEAN,
        original_tweet_id BIGINT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    )
    """
    cursor.execute(tweets_table_query)

    print("Tables created successfully")

In [4]:
from datetime import datetime

def convert_twitter_date_to_mysql(date_str):
    try:
        return datetime.strptime(date_str, '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError as e:
        print(f"Date conversion error: {e}")
        return None

Function to add records to the user table.

In [5]:
def add_user(cursor, user):
    formatted_date = convert_twitter_date_to_mysql(user['created_at'])
    
    query = """
        INSERT INTO Users (user_id, name, screen_name, location, followers_count,
                            friends_count, statuses_count, created_at) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
        ON DUPLICATE KEY UPDATE 
        name = IF(VALUES(created_at) > created_at, VALUES(name), name), 
        screen_name = IF(VALUES(created_at) > created_at, VALUES(screen_name), screen_name), 
        location = IF(VALUES(created_at) > created_at, VALUES(location), location), 
        followers_count = IF(VALUES(created_at) > created_at, VALUES(followers_count), followers_count), 
        friends_count = IF(VALUES(created_at) > created_at, VALUES(friends_count), friends_count), 
        statuses_count = IF(VALUES(created_at) > created_at, VALUES(statuses_count), statuses_count), 
        created_at = IF(VALUES(created_at) > created_at, VALUES(created_at), created_at)
    """

    values = (user['id'], user['name'], user['screen_name'], user['location'], user['followers_count'], 
              user['friends_count'], user['statuses_count'], formatted_date)

    cursor.execute(query, values)

Function to add records to tweet table.

In [6]:
def add_tweet(cursor, tweet, user_id):
    formatted_date = convert_twitter_date_to_mysql(tweet['created_at'])
    is_retweet = tweet['text'].startswith("RT")
    original_tweet_id = None

    # Defaults for all tweets
    is_quote_status = tweet.get('is_quote_status', False)
    quote_count = tweet.get('quote_count', 0)
    reply_count = tweet.get('reply_count', 0)
    retweet_count = tweet.get('retweet_count', 0)
    favorite_count = tweet.get('favorite_count', 0)

    if is_retweet and 'retweeted_status' in tweet:
        # For retweets, get these counts from the retweeted_status if available
        original_tweet_id = tweet['retweeted_status']['id']
        is_quote_status = tweet['retweeted_status'].get('is_quote_status', is_quote_status)
        quote_count = tweet['retweeted_status'].get('quote_count', quote_count)
        reply_count = tweet['retweeted_status'].get('reply_count', reply_count)
        retweet_count = tweet['retweeted_status'].get('retweet_count', retweet_count)
        favorite_count = tweet['retweeted_status'].get('favorite_count', favorite_count)

    query = """
        INSERT INTO Tweets (tweet_id, user_id, created_at, is_quote_status, quote_count, reply_count,
                    retweet_count, favorite_count, is_retweet, original_tweet_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
        user_id = IF(VALUES(created_at) > created_at, VALUES(user_id), user_id), 
        created_at = IF(VALUES(created_at) > created_at, VALUES(created_at), created_at),
        is_quote_status = IF(VALUES(created_at) > created_at, VALUES(is_quote_status), is_quote_status), 
        quote_count = IF(VALUES(created_at) > created_at, VALUES(quote_count), quote_count), 
        reply_count = IF(VALUES(created_at) > created_at, VALUES(reply_count), reply_count), 
        retweet_count = IF(VALUES(created_at) > created_at, VALUES(retweet_count), retweet_count), 
        favorite_count = IF(VALUES(created_at) > created_at, VALUES(favorite_count), favorite_count), 
        is_retweet = IF(VALUES(created_at) > created_at, VALUES(is_retweet), is_retweet), 
        original_tweet_id = IF(VALUES(created_at) > created_at, VALUES(original_tweet_id), original_tweet_id);
    """

    values = (tweet['id'], user_id, formatted_date, is_quote_status,
              quote_count, reply_count, retweet_count, favorite_count, 
              is_retweet, original_tweet_id)

    cursor.execute(query, values)


In [7]:
def process_dataset(filename, reset_tables=False):
    connection = create_database_connection()
    if connection is None:
        return
    else:
        cursor = connection.cursor()
        print("Connection established")

        if reset_tables:
            drop_and_create_tables(cursor)
            print("Fresh tables created successfully")

    with open(filename, "r") as file:
        batch_size = 1
        counter = 0
        for line in file:
            try:
                tweet = json.loads(line)
                user = tweet['user']
                
                add_user(cursor, user)
                add_tweet(cursor, tweet, user['id'])

                counter += 1
                if counter >= batch_size:
                    connection.commit()  # Committing after every record(due to the foreign key constraint)
                    counter = 0

            except json.JSONDecodeError:
                continue
            except Error as e:
                print("MySQL Error:", e)
                connection.rollback()

    cursor.close()
    connection.close()

In [11]:
print("Adding records from corona-2")
process_dataset(file_1, reset_tables=True)
print("Adding records from corona-3")
# For the second file, do not reset the tables
process_dataset(file_2, reset_tables=False)

Adding records from corona-2
Connection established
Tables created successfully
Fresh tables created successfully
Adding records from corona-3
Connection established


Now some queries on the tables created

In [12]:
def check_table_row_count(cursor, table_name):
    query = f"SELECT COUNT(*) FROM {table_name};"
    cursor.execute(query)
    return cursor.fetchone()[0]

def describe_table(cursor, table_name):
    query = f"DESCRIBE {table_name};"
    cursor.execute(query)
    return cursor.fetchall()

def preview_table_data(cursor, table_name, limit=5):
    query = f"SELECT * FROM {table_name} ORDER BY created_at desc LIMIT {limit};"
    cursor.execute(query)
    return cursor.fetchall()

def preview_tweet_data(cursor, table_name, limit=5):
    query = f"SELECT * FROM {table_name} ORDER BY retweet_count desc LIMIT {limit};"
    cursor.execute(query)
    return cursor.fetchall()

def date_range_tweets(cursor):
    query = "SELECT MIN(created_at), MAX(created_at) FROM Tweets;"
    cursor.execute(query)
    return cursor.fetchone()

def date_range_users(cursor):
    query = "SELECT MIN(created_at), MAX(created_at) FROM Users;"
    cursor.execute(query)
    return cursor.fetchone()

def check_for_duplicate_entries(cursor, table_name, column_name):
    query = f"""
        SELECT {column_name}, COUNT(*)
        FROM {table_name}
        GROUP BY {column_name} HAVING COUNT(*) > 1;
    """
    cursor.execute(query)
    return cursor.fetchall()

def get_tweet_with_id(cursor,tweetid):
    query=f"SELECT * FROM Tweets WHERE tweet_id={tweetid};"
    cursor.execute(query)
    return cursor.fetchall()

def pretty_print_results(title, results):
    print(f"\n{title}:")
    for row in results:
        print(", ".join(str(value) for value in row))

connection = create_database_connection()
if connection is not None:
    cursor = connection.cursor()
    try:
        # Checking table row count
        users_count = check_table_row_count(cursor, 'Users')
        tweets_count = check_table_row_count(cursor, 'Tweets')
        print(f"\nUsers count: {users_count}, Tweets count: {tweets_count}")

        # Describing table structure
        users_structure = describe_table(cursor, 'Users')
        tweets_structure = describe_table(cursor, 'Tweets')
        pretty_print_results("Users Structure", users_structure)
        pretty_print_results("Tweets Structure", tweets_structure)

        # Previewing data
        users_preview = preview_table_data(cursor, 'Users')
        tweets_preview = preview_table_data(cursor, 'Tweets')
        pretty_print_results("Users Preview", users_preview)
        pretty_print_results("Tweets Preview", tweets_preview)
        
        tweets_preview = preview_tweet_data(cursor, 'Tweets')
        pretty_print_results("Tweets Preview Ordered by retweet count", tweets_preview)

        # Date range in Tweets
        tweets_date_range = date_range_tweets(cursor)
        print(f"\nDate range in Tweets: {tweets_date_range[0]} to {tweets_date_range[1]}")
        
        # Date range in Users
        users_date_range = date_range_users(cursor)
        print(f"\nDate range in Users: {users_date_range[0]} to {users_date_range[1]}")

        # Check for duplicates
        duplicate_users = check_for_duplicate_entries(cursor, 'Users', 'user_id')
        duplicate_tweets = check_for_duplicate_entries(cursor, 'Tweets', 'tweet_id')
        pretty_print_results("Duplicate Users", duplicate_users)
        pretty_print_results("Duplicate Tweets", duplicate_tweets)
        
        #get tweet with particular id
        tweetid=1254059748556771329
        pretty_print_results(f"Tweet with id {tweetid} : ",get_tweet_with_id(cursor,tweetid))

    finally:
        cursor.close()
        connection.close()



Users count: 95654, Tweets count: 120400

Users Structure:
user_id, bigint, NO, PRI, None, 
name, varchar(255), YES, , None, 
screen_name, varchar(255), YES, , None, 
location, varchar(255), YES, , None, 
followers_count, int, YES, , None, 
friends_count, int, YES, , None, 
statuses_count, int, YES, , None, 
created_at, datetime, YES, , None, 

Tweets Structure:
tweet_id, bigint, NO, PRI, None, 
user_id, bigint, YES, MUL, None, 
created_at, datetime, YES, , None, 
is_quote_status, tinyint(1), YES, , None, 
quote_count, int, YES, , None, 
reply_count, int, YES, , None, 
retweet_count, int, YES, , None, 
favorite_count, int, YES, , None, 
is_retweet, tinyint(1), YES, , None, 
original_tweet_id, bigint, YES, , None, 

Users Preview:
1254058098861490176, Akif istaque Ali, IstaqueAkif, None, 0, 0, 1, 2020-04-25 14:42:30
1254057976958054401, djmkv, djmkv1, None, 0, 37, 1, 2020-04-25 14:41:37
1254057914433560577, POORANLALDUBEY, POORANLALDUBEY8, None, 0, 1, 1, 2020-04-25 14:41:31
12540564405

Some more analysis

In [41]:
def check_timezones_in_files(file_list):
    timezone_issues = {filename: False for filename in file_list}

    for filename in file_list:
        with open(filename, 'r') as file:
            for line in file:
                try:
                    tweet = json.loads(line)
                    created_at = tweet['user'].get('created_at')
                    if created_at:
                        timezone = datetime.strptime(created_at, '%a %b %d %H:%M:%S %z %Y').strftime('%z')
                        if timezone != '+0000':
                            timezone_issues[filename] = True
                            break
                except json.JSONDecodeError:
                    continue  
                except (ValueError, KeyError):
                    continue 

    return timezone_issues

timezone_issues = check_timezones_in_files([file_1, file_2])

for file, issue in timezone_issues.items():
    if issue:
        print(f"Timezone other than +0000 found in file: {file}")
    else:
        print(f"All 'created_at' dates in {file} have timezone +0000.")


All 'created_at' dates in /Users/manasarthak/Documents/DBMS Project/corona-out-2 have timezone +0000.
All 'created_at' dates in /Users/manasarthak/Documents/DBMS Project/corona-out-3 have timezone +0000.


In [15]:
def print_tweet_with_id(filename, tweet_id):
    with open(filename, 'r') as file:
        for line in file:
            try:
                data = json.loads(line)
                if data.get("id") == tweet_id:
                    print(json.dumps(data, indent=4))
                    break  # Exit the loop after finding the tweet
            except json.JSONDecodeError:
                continue  # Skip invalid lines

# File path and tweet ID
target_tweet_id = 1249408877554270208

# Call the function
print_tweet_with_id(file_1, target_tweet_id)


{
    "created_at": "Sun Apr 12 18:47:43 +0000 2020",
    "id": 1249408877554270208,
    "id_str": "1249408877554270208",
    "text": "RT @stayfrea_: ALERT\u203c\ufe0f\u203c\ufe0f\u203c\ufe0f\nThe corona virus can be spread through money. If you have any money at home, put on some gloves, put all the mon\u2026",
    "source": "<a href=\"https://mobile.twitter.com\" rel=\"nofollow\">Twitter Web App</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 944115288412442625,
        "id_str": "944115288412442625",
        "name": "m",
        "screen_name": "idslato",
        "location": null,
        "url": null,
        "description": null,
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 87,
        "friends_count": 109,
        "listed_co